In [7]:
import io
import fitz
import re
from PIL import Image, ImageDraw, ImageFont, ImageColor
import operator
import pandas as pd 
import numpy as np
import math
from tqdm import tqdm
import string 


# ROI Plotting

In [8]:
def plot_genus_blocks(page_df, draw, color = '#6c899e', w = 3):
    genus_list = page_df['genus'].unique()
    for c in page_df['col'].unique():
        for g in genus_list:
            temp_df = page_df[(page_df['col'] == c) & (page_df['genus'] == g)]
            g_x0 = temp_df['x0'].min()
            g_y0 = temp_df['y0'].min()
            g_x1 = temp_df['x1'].max()
            g_y1 = temp_df['y1'].max()

            draw.rectangle((g_x0, g_y0, g_x1, g_y1), fill=None, outline=ImageColor.getrgb(color), width = w)
        
def plot_epitet_blocks(page_df, draw, color = '#54081f', w = 2):
    epitet_list = page_df['epitet'].unique()
    for c in page_df['col'].unique():
        for s in epitet_list:
            temp_df = page_df[(page_df['col'] == c) & (page_df['epitet'] == s)]
            for g in temp_df['genus'].unique():
                    
                temp_g_df = temp_df[(temp_df['genus']) == g]
                s_x0 = temp_g_df['x0'].min()
                s_y0 = temp_g_df['y0'].min()
                s_x1 = temp_g_df['x1'].max()
                s_y1 = temp_g_df['y1'].max()

                draw.rectangle((s_x0, s_y0, s_x1, s_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_blocks(page_df, draw, color = '#4a3757', w = 2):
    block_list = page_df['block_no'].unique()
    for i in block_list:
        df_groupped = page_df[page_df['block_no'] == i]
        x0_arr = df_groupped['x0'].min()
        y0_arr = df_groupped['y0'].min()
        x1_arr = df_groupped['x1'].max()
        y1_arr = df_groupped['y1'].max()

        draw.rectangle((x0_arr, y0_arr, x1_arr, y1_arr), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_author_blocks(page_df, draw, color = '#a3a3a3', w = 1):
    epitet_list = page_df['epitet'].unique()
    for c in page_df['col'].unique():
        for s in epitet_list:
            temp_df = page_df[(page_df['col'] == c) & (page_df['epitet'] == s)]
            for g in temp_df['genus'].unique():
                temp_g_df = temp_df[(temp_df['genus'] == g) & (temp_df['word'] != s)]
                a_x0 = temp_g_df['x0'].min()
                a_y0 = temp_g_df['y0'].min()
                a_x1 = temp_g_df['x1'].max()
                a_y1 = temp_g_df['y1'].max()
                
                draw.rectangle((a_x0, a_y0, a_x1, a_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_sub_blocks(page_df, draw, color = '#ff6289', w = 1):
    sub_list = page_df[(~page_df['sub'].isnull())]['sub'].unique()
    for c in page_df['col'].unique():
        for s in sub_list:
            temp_df = page_df[(page_df['col'] == c) & (page_df['sub'] == s)]
            for b in temp_df['block_no'].unique():
                temp_b_df = temp_df[(temp_df['block_no'] == b)]
                for l in temp_b_df['line_no'].unique():
                    temp_l_df = temp_b_df[temp_b_df['line_no'] == l]
                    a_x0 = temp_l_df['x0'].min()
                    a_y0 = temp_l_df['y0'].min()
                    a_x1 = temp_l_df['x1'].max()
                    a_y1 = temp_l_df['y1'].max()
                
                    draw.rectangle((a_x0, a_y0, a_x1, a_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

# Vol3 Index

In [9]:
pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf"
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.pageCount)]
index = list(range(555, 583))

TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15

In [10]:
def valid(word):
    """
    valid words are words that are:
    - at least 2 characters
        - unless it's x (symbol for hybrid)
    """
    return not bool(re.search(r"[0-9]+[,.]?", word)) and (len(word) > 1 or word == 'x' or word == 'X' or word == '×')

def preprocessing(page_num):
    #group_by_indent
    """prune out invalid words"""
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    """reset word_no values (useful for cases where word that was originally at 0th index was pruned out"""
    for b in page_df['block_no'].unique():
        for l in page_df['line_no'].unique():
            page_df.loc[(page_df['line_no'] == l) & (page_df['block_no'] == b), 'word_no'] = np.arange(len(page_df[(page_df['line_no'] == l) & (page_df['block_no'] == b)]['word_no'])).astype(int) #this is slowww

    return page_df.reset_index()

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe

# ROI Plotting

In [11]:
def plot_genus_blocks(page_df, draw, color = '#6c899e', w = 3):
    genus_list = page_df['genus'].unique()
    for c in page_df['col'].unique():
        for g in genus_list:
            temp_df = page_df[(page_df['col'] == c) & (page_df['genus'] == g)]
            g_x0 = temp_df['x0'].min()
            g_y0 = temp_df['y0'].min()
            g_x1 = temp_df['x1'].max()
            g_y1 = temp_df['y1'].max()

            draw.rectangle((g_x0, g_y0, g_x1, g_y1), fill=None, outline=ImageColor.getrgb(color), width = w)
        
def plot_epitet_blocks(page_df, draw, color = '#54081f', w = 2):
    epitet_list = page_df['epitet'].unique()
    for c in page_df['col'].unique():
        for s in epitet_list:
            temp_df = page_df[(page_df['col'] == c) & (page_df['epitet'] == s)]
            for g in temp_df['genus'].unique():
                    
                temp_g_df = temp_df[(temp_df['genus']) == g]
                s_x0 = temp_g_df['x0'].min()
                s_y0 = temp_g_df['y0'].min()
                s_x1 = temp_g_df['x1'].max()
                s_y1 = temp_g_df['y1'].max()

                draw.rectangle((s_x0, s_y0, s_x1, s_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_blocks(page_df, draw, color = '#4a3757', w = 2):
    block_list = page_df['block_no'].unique()
    for i in block_list:
        df_groupped = page_df[page_df['block_no'] == i]
        x0_arr = df_groupped['x0'].min()
        y0_arr = df_groupped['y0'].min()
        x1_arr = df_groupped['x1'].max()
        y1_arr = df_groupped['y1'].max()

        draw.rectangle((x0_arr, y0_arr, x1_arr, y1_arr), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_author_blocks(page_df, draw, color = '#a3a3a3', w = 1):
    epitet_list = page_df['epitet'].unique()
    for c in page_df['col'].unique():
        for s in epitet_list:
            temp_df = page_df[(page_df['col'] == c) & (page_df['epitet'] == s)]
            for g in temp_df['genus'].unique():
                temp_g_df = temp_df[(temp_df['genus'] == g) & (temp_df['word'] != s)]
                a_x0 = temp_g_df['x0'].min()
                a_y0 = temp_g_df['y0'].min()
                a_x1 = temp_g_df['x1'].max()
                a_y1 = temp_g_df['y1'].max()
                
                draw.rectangle((a_x0, a_y0, a_x1, a_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

In [12]:
def is_genus(word):
    """
    A word in the index might be a genus if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - first letter upper case
        - all but first lowecase 
    in regex: ^[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+$
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"^[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+$"
    return re.search(regex, word)
    

def is_epithet(word):
    """
    A word in the index might be an epithet if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - all letters lowecase 
    in regex: ^[a-zàâäèéêëîïôœùûüÿç]+[-]?[a-zàâäèéêëîïôœùûüÿç]+$
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"^[a-zàâäèéêëîïôœùûüÿç]+[-]?[a-zàâäèéêëîïôœùûüÿç]+$"
    return re.search(regex, word)

In [13]:
def process_df(page_num, genus = np.NaN, genus_block_no = np.NaN):
    def initiate_groups(row):
        #return row
        x_0, y_0, x_1, y_1 = row['x0'], row['y0'], row['x1'], row['y1']
        word_no = row['word_no']
        word = row['word']
        alphnum_word = ''.join(e for e in word if e.isalnum())
        alph_word = ''.join(e for e in word if e.isalpha())
        
        if row['line_no'] == 0 and (word.lower() == 'nouvelle' or word.lower() == 'flore'):
            return 
        if (word_no == 0) and (not alphnum_word.isnumeric()) and alph_word:
            #word_no == 0 => the word is a Family, Genus, Species
            new_group = True
            for g in indent_groups:
                g_arr = np.array(g)[:,0]
                if (x_0, y_0, x_1, y_1)[0] <= np.mean(g_arr) + indent_err and (x_0, y_0, x_1, y_1)[0] >= np.mean(g_arr) - indent_err:
                    g.append((x_0, y_0, x_1, y_1))
                    new_group = False
            if new_group:
                indent_groups.append([(x_0, y_0, x_1, y_1)])

    def get_indent_group(row):
        x_0, y_0, x_1, y_1 = row['x0'], row['y0'], row['x1'], row['y1']
        word_no = row['word_no']
        word = row['word']
        alphnum_word = ''.join(e for e in word if e.isalnum())
        
        if (len(word) > 1) and ((not alphnum_word.isnumeric()) and (word_no == 0)): 
            for g_i, g in enumerate(indent_groups):
                g_arr = np.array(g)[:,0]
                if (x_0, y_0, x_1, y_1)[0] <= np.mean(g_arr) + indent_err and (x_0, y_0, x_1, y_1)[0] >= np.mean(g_arr) - indent_err:
                    return g_i

    def get_col(row): 
        x_0, y_0, x_1, y_1 = row['x0'], row['y0'], row['x1'], row['y1']
        return int(x_0 > ((x_min + x_max) / 2))

    def n_leftmost_indent(indent_groups, n):
        indent_means = []
        for g_i, g in enumerate(indent_groups):
            g_arr = np.array(g)[:,0]
            indent_means.append([g_i, np.mean(g_arr)])

        n_smallest = indent_means.sort(key = lambda x : x[1])[:n] #n smallest
        return [el[0] for el in n_smallest]

    def process_col(row):
        nonlocal genus
        nonlocal x_0, y_0, x_1, y_1 
        nonlocal epitet
        nonlocal genus_block_no
        word_no = row['word_no']
        block_no = row['block_no']
        line_no = row['line_no']
        word = row['word']

        row['epitet'] = np.NaN
        row['genus'] = np.NaN
        
        alph_word = ''.join(e for e in word if e.isalpha())

        if line_no == 0 and (word.lower() == 'nouvelle' or word.lower() == 'flore'):
            return row

        if (not (word.isupper() and word_no == 0)) and len(word) > 1  and alph_word:
            if word_no == 0: #epitet, or genus
                x_0, y_0, x_1, y_1 = row['x0'], row['y0'], row['x1'], row['y1']

            alphnum_word = ''.join(e for e in word if e.isalnum())
            if (not alphnum_word.isnumeric()): 
                if  x_0 <= g_x0 + indent_err and x_0 >= g_x0 - indent_err:
                    if word_no == 0:
                        genus = word
                        row['genus'] = genus
                        genus_block_no = row['block_no']
                    elif word_no != 0 and line_no == 0 and block_no == genus_block_no: #info on same line as genus
                        #epitet = word
                        #print(genus, alphnum_word)
                        row['epitet'] = np.NaN
                        row['genus'] = genus
                        #row['author']"""
                        row['sub'] = np.NaN
                    else:
                        row['epitet'] = epitet
                        row['genus'] = genus
                        row['sub'] = np.NaN
                elif x_0 <= s_x0 + indent_err and x_0 >= s_x0 - indent_err:
                    if word_no == 0:
                        epitet = word
                    row['epitet'] = epitet
                    row['genus'] = genus
                    row['sub'] = np.NaN
                elif word_no == 0 and (word == 'var.' or word == 'subsp.' or word == 'x' or word == 'X'): #assuming at least genus line exists
                    row['epitet'] = epitet
                    row['genus'] = genus
                    row['sub'] = word

        return row

    #page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    #page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    
    page_df = preprocessing(page_num)
    indent_groups = []
    indent_err = 15

    x_min = page_df['x0'].min()
    y_min = page_df['y0'].min()
    x_max = page_df['x1'].max()
    y_max = page_df['y1'].max()
    
    page_df.apply(initiate_groups, axis = 1)
    page_df['indent_group'] = page_df.apply(get_indent_group, axis = 1)

    page_df['col'] = page_df.apply(get_col, axis = 1)
    
    x_0, y_0, x_1, y_1, epitet = np.NaN, np.NaN, np.NaN, np.NaN, np.NaN
    for c in page_df['col'].unique():
        
        # what if just one largest? what would that even mean? Hm ... (like what if it's all)
        col_df = page_df[(page_df['col'] == c)]
        col_indent_groups =  list(col_df[~col_df['indent_group'].isnull()]['indent_group'].unique())
        #n_leftmost = n_leftmost_indent(col_indent_groups, 2)
        #col_df[col_df['indent_group'] == s_indent]
        #if len(n_leftmost) == 2:

        s_x0, g_x0 = float('inf'), float('inf')
        s_indent, g_indent = -1, -1

        for g in col_indent_groups:
            mean_x0 = col_df[col_df['indent_group'] == g]['x0'].mean()
            if g_x0 > mean_x0:
                s_indent, g_indent = g_indent, g
                s_x0, g_x0 = g_x0, mean_x0 
            elif s_x0 > mean_x0: #and g_x0 <= mean_x0
                s_indent = g
                s_x0 = mean_x0

        #if col_df[col_df['indent_group'] == s_indent]['word'].str.contains('var.|subsp.').any():
        
        #print(genus, s_x0, g_x0)


        species_indent_df = col_df[col_df['indent_group'] == s_indent]
        if (species_indent_df['word'] == 'var.').any() or (species_indent_df['word'] == 'subsp.').any():
            s_x0, g_x0 = g_x0, float('inf')
            s_indent, g_indent = g_indent, -1
            #print("no genus in a column of page", page_num)
        if (s_x0  < g_x0):
            s_x0, g_x0 = g_x0, s_x0
            s_indent, g_indent = g_indent, s_indent

        if s_indent == -1:
            s_x0, g_x0 = g_x0, float('inf')
            s_indent, g_indent = g_indent, -1

        col_df = col_df.apply(process_col, axis = 1)
        page_df.loc[col_df.index, ['genus', 'epitet', 'sub']] = col_df.loc[col_df.index, ['genus', 'epitet', 'sub']]
        #print("g_x0, s_x0, g_indent, s_indent:", g_x0, s_x0, g_indent, s_indent)
    #print("genus", genus)

    return page_df, genus

In [14]:
def get_author(page_df):
    #pruned_df = page_df[(~page_df['genus'].isnull())].reset_index()
    epitet_names = page_df[~page_df['epitet'].isnull()]['epitet'].unique()
    genus_names = page_df[~page_df['genus'].isnull()]['genus'].unique()
    for i in range(len(page_df['x0'])):
        word = page_df.loc[i, 'word']
        if word in epitet_names:
            #print(word, i)
            s = page_df.loc[i, 'word']
            g = page_df.loc[i, 'genus']
            sub_df = page_df[(page_df['genus'] == g) & (page_df['epitet'] == s) & (page_df['word'] != s)]
            
            merged = sub_df.groupby('epitet')['word'].agg(' '.join).reset_index()
            
            concat_str = np.NaN
            if len(merged.index):
                concat_str = merged['word'].item()
            
            page_df.loc[i, 'author'] = concat_str

        if word in genus_names:
            g = page_df.loc[i, 'genus']
            g_block_no = page_df.loc[i, 'block_no']
            sub_df = page_df[(page_df['genus'] == g) & (page_df['block_no'] == g_block_no) & (page_df['word_no'] != 0) & (page_df['line_no'] == 0)]
            merged = sub_df.groupby('genus')['word'].agg(' '.join).reset_index()
            concat_str = np.NaN
            if len(merged.index):
                #print(g, word)
                concat_str = merged['word'].item()
            page_df.loc[i, 'author'] = concat_str

In [17]:
genus = np.NaN
result_ims = []
df_list = []

for page_num in tqdm(index):
    page_df, genus = process_df(page_num, genus)
    get_author(page_df)
    
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)

    plot_genus_blocks(page_df, draw, w = 4)
    plot_epitet_blocks(page_df, draw, w = 3)
    plot_author_blocks(page_df, draw, w = 2)

    df_list.append(page_df)
    result_ims.append(image)
    #break
result_ims[0].save('../output/index/PDF/vol3_with_preprocess.pdf',save_all=True, append_images=result_ims[1:])


100%|██████████| 28/28 [01:10<00:00,  2.50s/it]


In [13]:
df = pd.concat(df_list, axis = 0)
#df.to_csv('../output/index/CSV/vol2.csv')

In [16]:
df[~df['sub'].isnull()]

,in_x0,in_y0,in_x1,in_y1,word,block_no,line_no,word_no,page_num,x0,y0,x1,y1,indent_group,col,genus,epitet,sub,author
96,52.320000,492.379181,66.466461,501.812378,var.,30,0,0,721,217.999999,2051.579920,276.943588,2090.884908,3.0,0,Nigella,arvensis,var.,NaN
99,52.320000,501.979218,66.466461,511.412415,var.,30,1,0,721,217.999999,2091.580073,276.943588,2130.885061,3.0,0,Nigella,arvensis,var.,NaN
102,52.560001,511.579193,66.367104,521.012390,var.,30,2,0,721,219.000006,2131.579971,276.529598,2170.884959,3.0,0,Nigella,arvensis,var.,NaN


In [20]:
author_pruned_df = df[(~df['author'].isnull()) | (df['word'] == df['genus'])]
simple_genus_species_author = author_pruned_df[["genus", "epitet", "author"]]
#simple_genus_species_author.to_csv('../output/index/CSV/vol2_simplified.csv', index = False)